In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Fine tuning SpeechT5 model focused on Technical Term

**install required packages





In [ ]:
!pip install datasets soundfile speechbrain
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 40.0 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-4rohjoxz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-4rohjoxz
  Resolved https://github.com/huggingface/transformers.git to commit 343c8cb86f2ab6a51e7363ee11f69afb1c9e839e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metada

**GPU**

In [ ]:
!nvidia-smi

Wed Oct 23 11:29:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

#Load the model

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

model.safetensors:   0%|          | 0.00/585M [00:00<?, ?B/s]

#Load the dataset

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset(
    "keithito/lj_speech", split="train"
)

README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

lj_speech.py:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

The repository for keithito/lj_speech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/keithito/lj_speech.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/13100 [00:00<?, ? examples/s]

It's important to set the sampling rate of the audio data to 16 kHz, which is what SpeechT5 expects.

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

Let's quickly check how many examples are in this dataset.

In [ ]:
len(dataset)

13100

In [ ]:
print(dataset)

Dataset({
    features: ['id', 'audio', 'file', 'text', 'normalized_text'],
    num_rows: 13100
})


In [ ]:
# Calculate the number of rows for the full dataset
full_size = len(dataset)

# Select the full dataset
dataset = dataset.select(range(full_size))
print(dataset)

Dataset({
    features: ['id', 'audio', 'file', 'text', 'normalized_text'],
    num_rows: 13100
})


**Clean up the text**

In [ ]:
tokenizer = processor.tokenizer

In [ ]:
dataset[2:5]

{'id': ['LJ001-0003', 'LJ001-0004', 'LJ001-0005'],
 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0003.wav',
   'array': array([-0.00404182, -0.00031247, -0.00158895, ...,  0.00022209,
           0.00031626, -0.00021122]),
   'sampling_rate': 16000},
  {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0004.wav',
   'array': array([ 1.84258504e-04, -6.89462904e-05, -2.43774717e-04, ...,
          -5.48143697e-04, -6.62148464e-04, -6.06632268e-04]),
   'sampling_rate': 16000},
  {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0005.wav',
   'array': array([0.00032222, 0.00043797, 0.00043948, ..., 0.00040887, 0.00078317,
          0.        ]),
   'sampling_rate': 16000}

In [ ]:
def extract_all_chars(batch):
    # Assuming 'text' is the correct column name containing the transcriptions
    all_text = " ".join(batch["text"])  # Changed "transcription" to "text"
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [ ]:
dataset_vocab - tokenizer_vocab

{' ',
 '$',
 '&',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '£',
 'à',
 'â',
 'è',
 'ü',
 '’',
 '“',
 '”'}

In [ ]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation (except apostrophes)
    text = re.sub(r'[^\w\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['text'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])

Map:   0%|          | 0/13100 [00:00<?, ? examples/s]

{'id': ['LJ001-0003', 'LJ001-0004', 'LJ001-0005'], 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0003.wav', 'array': array([-0.00404182, -0.00031247, -0.00158895, ...,  0.00022209,
        0.00031626, -0.00021122]), 'sampling_rate': 16000}, {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0004.wav', 'array': array([ 1.84258504e-04, -6.89462904e-05, -2.43774717e-04, ...,
       -5.48143697e-04, -6.62148464e-04, -6.06632268e-04]), 'sampling_rate': 16000}, {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0005.wav', 'array': array([0.00032222, 0.00043797, 0.00043948, ..., 0.00040887, 0.00078317,
       0.        ]), 'sampling_rate': 16000}], 'file': ['/root/.cache/huggin

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [ ]:
dataset_vocab - tokenizer_vocab

{' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'à', 'â', 'è', 'ü'}

**Speaker embeddings**

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

<ipython-input-19-f28baad2ea60>:3: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [ ]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [ ]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [ ]:
processed_example["speaker_embeddings"].shape

(512,)

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [ ]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 300

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

Filter:   0%|          | 0/13100 [00:00<?, ? examples/s]

13100

**Preparing the dataset**
#Train/test split

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        ) # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
features = [
    dataset["train"][0],
    dataset["train"][1],
    dataset["train"][20],
]

batch = data_collator(features)

In [ ]:
{k:v.shape for k,v in batch.items()}

{'input_ids': torch.Size([3, 122]),
 'attention_mask': torch.Size([3, 122]),
 'labels': torch.Size([3, 630, 80]),
 'speaker_embeddings': torch.Size([3, 512])}

In [ ]:
model.config.use_cache = False

In [ ]:
from functools import partial
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_English",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=1500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

<ipython-input-33-3e9fd9c3f140>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
100,3.772000,0.420103
200,3.591200,0.405249
300,3.460400,0.394766
400,3.389400,0.390611
500,3.373700,0.386506
600,3.362800,0.385102
700,3.323600,0.382134
800,3.306000,0.381117
900,3.285900,0.379681
1000,3.266300,0.376319


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2816: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 1876}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10

TrainOutput(global_step=1500, training_loss=3.39983762105306, metrics={'train_runtime': 4837.6694, 'train_samples_per_second': 9.922, 'train_steps_per_second': 0.31, 'total_flos': 5483826441583776.0, 'train_loss': 3.39983762105306, 'epoch': 4.580152671755725})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/DeepDiveDev/speecht5_finetuned_English/commit/8bbc92b5968125e1ff51b8fea16e90aa40c5f267', commit_message='End of training', commit_description='', oid='8bbc92b5968125e1ff51b8fea16e90aa40c5f267', pr_url=None, pr_revision=None, pr_num=None)

#Dataset on Technical Term

In [ ]:
!pip install datasets
import pandas as pd
from datasets import Dataset, load_dataset

# Load your dataset from Excel file using pd.read_excel
df = pd.read_excel('/content/drive/MyDrive/TTS_Eng/TTS-English.xlsx')
# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00


In [ ]:
len(dataset)

133

In [ ]:
!pip install transformers
from transformers import AutoProcessor

# Assuming you want to use a specific processor, replace "facebook/wav2vec2-base-960h" with the desired model
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")

tokenizer = processor.tokenizer

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset

# Load your dataset from Excel file using pd.read_excel
df = pd.read_excel('/content/drive/MyDrive/TTS_Eng/TTS-English.xlsx')
# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

tokenizer = processor.tokenizer

def extract_all_chars(batch):
    text_examples = [text for text in batch["Text Example"] if text is not None]
    all_text = " ".join(text_examples)
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

# Only remove unnecessary columns, but keep the text column
vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    # Instead of removing all columns, specify only the ones you want to remove.
    # Replace with the actual columns you want to remove if needed.
    remove_columns=[col for col in dataset.column_names if col != "transcript"],
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k,_ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

In [ ]:
dataset_vocab - tokenizer_vocab

{' ',
 '"',
 '%',
 ',',
 '-',
 '.',
 '0',
 '2',
 '4',
 '5',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '’'}

In [ ]:
replacements = [
    ('à', 'a'),
    ('ç', 'c'),
    ('è', 'e'),
    ('ë', 'e'),
    ('í', 'i'),
    ('ï', 'i'),
    ('ö', 'o'),
    ('ü', 'u'),
    ('’', "'"),  # Replacing curly apostrophe with a standard one
    ('%', ''),   # Option to remove the percentage symbol
    ('0', '0'),  # Keep 0 as it is (no change)
    ('2', '2'),  # Keep 2 as it is (no change)
    ('4', '4'),  # Keep 4 as it is (no change)
    ('5', '5'),  # Keep 5 as it is (no change)
    (' ', ' ')   # Ensure spaces remain unchanged
]

def cleanup_text(inputs):
    text_column_name = "Text Example"  # Update with the correct column name from your dataset
    # Check if the value is not None using the correct column name
    if inputs[text_column_name] is not None:
        for src, dst in replacements:
            # Update this line to use the correct column name
            inputs[text_column_name] = inputs[text_column_name].replace(src, dst)
    return inputs

# Apply the function to the dataset
dataset = dataset.map(cleanup_text)

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

In [ ]:
!pip install speechbrain
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 30.3 MB/s eta 0:00:00


<ipython-input-19-e1b7115d85ce>:4: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [ ]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [ ]:
def prepare_dataset(example):
    # Check if 'audio' key exists before accessing it
    if 'audio' in example:
        audio = example["audio"]
    else:
        # Handle the case where 'audio' key is missing
        # This could be raising an error, skipping the example, or using a default value
        # For example, to skip the example:
        print("Skipping example due to missing 'audio' key")
        return None

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 'audio' key
Skipping example due to missing 

In [ ]:
def is_not_too_long(example):
  input_ids = example.get('Text Example')  # Safely access 'Text Example'
  if input_ids is not None:
    input_length = len(input_ids)
    return input_length < 300
  else:
    return False  # Skip examples with missing or None 'Text Example'


dataset = dataset.filter(is_not_too_long) # No need to specify input_columns
len(dataset)

Filter:   0%|          | 0/133 [00:00<?, ? examples/s]

105

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )# not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
from transformers import SpeechT5ForTextToSpeech

# Define the checkpoint variable with the desired model name or path
checkpoint = "microsoft/speecht5_tts"  # Example checkpoint for SpeechT5 text-to-speech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/speecht5_finetuned_English",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=1500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install datasets transformers[torch] accelerate
!pip install sklearn

from sklearn.model_selection import train_test_split
import datasets
import numpy as np
from transformers import Seq2SeqTrainer # Import Seq2SeqTrainer

# Convert the Hugging Face Dataset to a NumPy array or list
# before splitting. Assuming 'dataset' is a Hugging Face Dataset
dataset_np = dataset[:] # If dataset is a DatasetDict, select the desired split, e.g., dataset["train"][:]
#dataset_np = np.array(dataset) #If you need a NumPy array specifically

# Split the NumPy array using train_test_split
train_indices, test_indices = train_test_split(np.arange(len(dataset_np)), test_size=0.2, random_state=42) # Adjust test_size and random_state as needed

# Create new Hugging Face Datasets using the split indices
train_dataset = dataset.select(train_indices)
test_dataset = dataset.select(test_indices)

# Now you can pass these datasets to the trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,  # Use train_dataset instead of dataset["train"]
    eval_dataset=test_dataset,  # Use test_dataset instead of dataset["test"]
    data_collator=data_collator,
    tokenizer=processor,
)

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
!pip install datasets transformers[torch] accelerate
!pip install sklearn

from sklearn.model_selection import train_test_split
import datasets
import numpy as np

# Assuming 'dataset' is your Hugging Face Dataset
# Rename or add the required columns to match the model's expected input.
# For example, if your dataset has a column named 'text' containing the target text,
# rename it to 'decoder_input_values':
dataset = dataset.rename_column('Text Example', 'decoder_input_values')
# Similarly, rename or add other necessary columns like 'input_values', 'attention_mask', and 'decoder_attention_mask'.

# Convert the Hugging Face Dataset to a NumPy array or list
# before splitting. Assuming 'dataset' is a Hugging Face Dataset
dataset_np = dataset[:] # If dataset is a DatasetDict, select the desired split, e.g., dataset["train"][:]
#dataset_np = np.array(dataset) #If you need a NumPy array specifically

# Split the NumPy array using train_test_split
train_indices, test_indices = train_test_split(np.arange(len(dataset_np)), test_size=0.2, random_state=42) # Adjust test_size and random_state as needed

# Create new Hugging Face Datasets using the split indices
train_dataset = dataset.select(train_indices)
test_dataset = dataset.select(test_indices)

# You might need to define a data collator to prepare the data for the model:
def data_collator(features):
    # Implement logic to collate data based on the columns present in your dataset
    # and the format required by the SpeechT5ForTextToSpeech model.
    # This might involve padding, creating attention masks, etc.
    # You can refer to the documentation for examples of data collators:
    # https://huggingface.co/docs/transformers/main_classes/data_collator
    pass  # Replace with your data collation logic

# Now you can pass these datasets to the trainer
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,  # Use train_dataset instead of dataset["train"]
    eval_dataset=test_dataset,  # Use test_dataset instead of dataset["test"]
    data_collator=data_collator,
    tokenizer=processor, # Assuming 'processor' is your SpeechT5Processor
)

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from transformers import SpeechT5HifiGan
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

In [ ]:
print(example["decoder_input_values"])  # Check if the values exist and are valid

When you connect to an external service, make sure the API returns the correct status codes.


In [ ]:
print(processed_example["input_ids"])  # Ensure input_ids are not None
print(processed_example["attention_mask"])  # Ensure attention_mask is not None

tensor([[ 4, 38, 11,  5,  9,  4, 22,  8, 16,  4, 17,  8,  9,  9,  5, 17,  6,  4,
          6,  8,  4,  7,  9,  4,  5, 37,  6,  5, 13,  9,  7, 15,  4, 12,  5, 13,
         27, 10, 17,  5, 23,  4, 18,  7, 28,  5,  4, 12, 16, 13,  5,  4,  6, 11,
          5,  4, 33, 49, 30,  4, 13,  5,  6, 16, 13,  9, 12,  4,  6, 11,  5,  4,
         17,  8, 13, 13,  5, 17,  6,  2]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])


#Interence

In [ ]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained("DeepDiveDev/speecht5_finetuned_English")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Example MOS collection
mos_scores = {
    'sentence_1.wav': [4, 5, 4],
    'sentence_2.wav': [3, 4, 4],
    # Add more scores for each audio sample
}

# Calculate the average MOS score for each audio
average_mos = {file: sum(scores)/len(scores) for file, scores in mos_scores.items()}
print(average_mos)

{'sentence_1.wav': 4.333333333333333, 'sentence_2.wav': 3.6666666666666665}


In [ ]:
import re

number_words = {
    0: "zero", 1: "one", 2: "two", 3: "three", 4: "four", 5: "five", 6: "six", 7: "seven", 8: "eight", 9: "nine",
    10: "ten", 11: "eleven", 12: "twelve", 13: "thirteen", 14: "fourteen", 15: "fifteen", 16: "sixteen", 17: "seventeen",
    18: "eighteen", 19: "nineteen", 20: "twenty", 30: "thirty", 40: "forty", 50: "fifty", 60: "sixty", 70: "seventy",
    80: "eighty", 90: "ninety", 100: "hundred", 1000: "thousand"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + ("-" + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        return (number_words[hundreds] + " hundred" if hundreds > 1 else "hundred") + (" and " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        return number_to_words(thousands) + " thousand" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " million" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " billion" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Find the numbers and change with words.
    result = re.sub(r'\b\d+\b', replace, text)

    return result

In [ ]:
import re

# Dictionary of technical words and their spoken equivalents (each letter separated)
technical_words = {
    "API": "A P I",
    "CUDA": "C U D A",
    "OAuth": "O Auth",
    "LLM": "L L M",
    "HTTP": "H T T P",
    "HTTPS": "H T T P S",
    "URL": "U R L",
    "SQL": "S Q L",
    "JSON": "J S O N",
    "XML": "X M L",
    "REST": "R E S T",
    "JWT": "J W T",
    "FTP": "F T P",
    "SSH": "S S H",
    "GPU": "G P U",
    "CPU": "C P U",
    "IP": "I P",
    "RAM": "R A M",
    "ROM": "R O M",
    "ID": "I D",
    "UID": "U I D",
    "UUID": "U U I D",
    "NLP": "N L P",
    "ML": "M L",
    "AI": "A I",
    "IoT": "I O T",
    "VPN": "V P N",
    "DNS": "D N S",
    "SMTP": "S M T P",
    "KNN": "K N N",
    "CNN": "C N N",
    "LSTM": "L S T M",
    "GRU": "G R U",
    # Add more technical terms with letters separated
}

def technical_words_to_speech(word):
    # Convert technical words to their spoken form, with each letter separated by spaces
    return technical_words.get(word, word)  # If not in the dictionary, return the original word

def replace_technical_words_with_speech(text):
    # Split the text into words and check for technical terms
    words = re.findall(r'\b\w+\b', text)

    result = []
    for word in words:
        # Convert each word to its spoken form if it's a technical term
        spoken_form = technical_words_to_speech(word)
        result.append(spoken_form)

    return " ".join(result)

# Example usage
text = "I will use an API with OAuth and CUDA to train the LLM model on a GPU."
spoken_text = replace_technical_words_with_speech(text)
print(spoken_text)

In [ ]:
inputs = processor(text=final_text, return_tensors="pt")

In [ ]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [ ]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'output.wav')
sf.write('output.wav', speech.numpy(), 16000)